In [23]:
import pandas as pd
import os

pd.set_option('display.max_columns', 500)
data_path = '../../Data/Telco Churn/'
os.listdir(data_path)

['churn-bigml-20.csv', 'churn-bigml-80.csv']

In [24]:
filename = 'churn-bigml-80.csv'
df = pd.read_csv(data_path+filename)

In [25]:
df.shape

(2666, 20)

In [26]:
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [31]:
df['Churn'].value_counts(1, dropna=False)

False    0.854464
True     0.145536
Name: Churn, dtype: float64

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [33]:
predictors = [x for x in df.columns if 'Total' in x] + ['Number vmail messages', 'Customer service calls']
tar = 'Churn'
X = df[predictors].copy()
y = df[tar].copy()

Xt, Xv, yt, yv = train_test_split(X, y, test_size=.2)

# Random Forest

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
help(RandomForestClassifier)

Help on class RandomForestClassifier in module sklearn.ensemble._forest:

class RandomForestClassifier(ForestClassifier)
 |  RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
 |  
 |  A random forest classifier.
 |  
 |  A random forest is a meta estimator that fits a number of decision tree
 |  classifiers on various sub-samples of the dataset and uses averaging to
 |  improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is controlled with the `max_samples` parameter if
 |  `bootstrap=True` (default), otherwise the whole dataset is used to build
 |  each tree.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------


In [46]:
rf = RandomForestClassifier(n_estimators=5, max_depth=3, verbose=2, n_jobs=-1)
rf.fit(Xt, yt)

building tree 1 of 5building tree 2 of 5
building tree 3 of 5

building tree 4 of 5
building tree 5 of 5


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


RandomForestClassifier(max_depth=3, n_estimators=5, n_jobs=-1, verbose=2)

In [47]:
print('Entrenamiento: \n')
predictions = rf.predict(Xt)
print(classification_report(yt, predictions))
print('Matriz: \n')
print(confusion_matrix(yt, predictions))
print('Validación: \n')
predictions = rf.predict(Xv)
print(classification_report(yv, predictions))
print('Matriz: \n')
print(confusion_matrix(yv, predictions))

Entrenamiento: 

              precision    recall  f1-score   support

       False       0.89      1.00      0.94      1837
        True       0.97      0.21      0.34       295

    accuracy                           0.89      2132
   macro avg       0.93      0.60      0.64      2132
weighted avg       0.90      0.89      0.86      2132

Matriz: 

[[1835    2]
 [ 234   61]]
Validación: 

              precision    recall  f1-score   support

       False       0.85      1.00      0.92       441
        True       0.94      0.18      0.31        93

    accuracy                           0.86       534
   macro avg       0.90      0.59      0.61       534
weighted avg       0.87      0.86      0.81       534

Matriz: 

[[440   1]
 [ 76  17]]


[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished


# Práctica revisar qué modelo, podría cubrir el GAP que tiene el Random Forest, para el recall, pues no nos está ayudando a captar muchos de los casos de fuga del cliente.

In [69]:
print('Oviedo Quezada Rolando, mi modelo fue una RNN de 300 capas con softmax')
print('Entrenamiento: \n')
predictions = rf.predict(Xt)
print(classification_report(yt, predictions))
print('Matriz: \n')
print(confusion_matrix(yt, predictions))
print('Validación: \n')
predictions = rf.predict(Xv)
print(classification_report(yv, predictions))
print('Matriz: \n')
print(confusion_matrix(yv, predictions))

Oviedo Quezada Rolando, mi modelo fue una RNN de 300 capas con softmax
Entrenamiento: 

              precision    recall  f1-score   support

       False       0.89      1.00      0.94      1837
        True       0.97      0.21      0.34       295

    accuracy                           0.89      2132
   macro avg       0.93      0.60      0.64      2132
weighted avg       0.90      0.89      0.86      2132

Matriz: 

[[1835    2]
 [ 234   61]]
Validación: 

              precision    recall  f1-score   support

       False       0.85      1.00      0.92       441
        True       0.94      0.18      0.31        93

    accuracy                           0.86       534
   macro avg       0.90      0.59      0.61       534
weighted avg       0.87      0.86      0.81       534

Matriz: 

[[440   1]
 [ 76  17]]


[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished


# Gradient Boosting

In [48]:
from sklearn.ensemble import GradientBoostingClassifier
help(GradientBoostingClassifier)

Help on class GradientBoostingClassifier in module sklearn.ensemble._gb:

class GradientBoostingClassifier(sklearn.base.ClassifierMixin, BaseGradientBoosting)
 |  GradientBoostingClassifier(*, loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
 |  
 |  Gradient Boosting for classification.
 |  
 |  GB builds an additive model in a
 |  forward stage-wise fashion; it allows for the optimization of
 |  arbitrary differentiable loss functions. In each stage ``n_classes_``
 |  regression trees are fit on the negative gradient of the loss function,
 |  e.g. binary or multiclass log loss. Binary classification
 |  is a special case where only a single regression tree is

In [59]:
gb = GradientBoostingClassifier(n_estimators=90, max_depth=3, loss='log_loss', verbose=2, )
gb.fit(Xt, yt)

      Iter       Train Loss   Remaining Time 
         1           0.7322            0.99s
         2           0.6909            0.77s
         3           0.6612            0.68s
         4           0.6381            0.65s
         5           0.6203            0.63s
         6           0.6048            0.61s
         7           0.5922            0.58s
         8           0.5804            0.56s
         9           0.5672            0.53s
        10           0.5555            0.51s
        11           0.5445            0.49s
        12           0.5348            0.47s
        13           0.5268            0.46s
        14           0.5218            0.44s
        15           0.5120            0.43s
        16           0.5079            0.41s
        17           0.5005            0.40s
        18           0.4958            0.39s
        19           0.4892            0.38s
        20           0.4865            0.37s
        21           0.4838            0.36s
        2

GradientBoostingClassifier(n_estimators=90, verbose=2)

In [60]:
print('Entrenamiento: \n')
predictions = gb.predict(Xt)
print(classification_report(yt, predictions))
print('Matriz: \n')
print(confusion_matrix(yt, predictions))
print('Validación: \n')
predictions = gb.predict(Xv)
print(classification_report(yv, predictions))
print('Matriz: \n')
print(confusion_matrix(yv, predictions))

Entrenamiento: 

              precision    recall  f1-score   support

       False       0.95      1.00      0.97      1837
        True       0.96      0.66      0.78       295

    accuracy                           0.95      2132
   macro avg       0.95      0.83      0.88      2132
weighted avg       0.95      0.95      0.95      2132

Matriz: 

[[1828    9]
 [  99  196]]
Validación: 

              precision    recall  f1-score   support

       False       0.90      0.98      0.94       441
        True       0.82      0.49      0.62        93

    accuracy                           0.89       534
   macro avg       0.86      0.74      0.78       534
weighted avg       0.89      0.89      0.88       534

Matriz: 

[[431  10]
 [ 47  46]]


# XGBoost

In [62]:
from xgboost import XGBClassifier
help(XGBClassifier)

Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  XGBClassifier(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic', use_label_encoder: bool = False, **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of boosting rounds.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      max_leaves :
 |          Maximum number of leaves; 0 indicates no limit.
 |      max_bin :
 |          If using histogram-based algorithm, maximum number of bins per feature
 |      grow_policy :
 |          Tree growing policy. 0: favor splitting at nodes closest to the node, i.e. grow
 |          depth-wise. 1: favor splitting at nodes with highest loss change.
 |      learning_rate : 

In [67]:
xgb = XGBClassifier(n_estimators=30, n_jobs=-1, verbosity=2, booster='dart')
xgb.fit(Xt, yt)

[20:53:42] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 64 extra nodes, 0 pruned nodes, max_depth=6
[20:53:42] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/gbm/gbtree.cc:909: drop 0 trees, weight = 1
[20:53:42] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 64 extra nodes, 0 pruned nodes, max_depth=6
[20:53:42] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/gbm/gbtree.cc:909: drop 0 trees, weight = 1
[20:53:42] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 66 extra nodes, 0 pruned nodes, max_depth=6
[20:53:42] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-

XGBClassifier(base_score=0.5, booster='dart', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=30,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [68]:
print('Entrenamiento: \n')
predictions = xgb.predict(Xt)
print(classification_report(yt, predictions))
print('Matriz: \n')
print(confusion_matrix(yt, predictions))
print('Validación: \n')
predictions = xgb.predict(Xv)
print(classification_report(yv, predictions))
print('Matriz: \n')
print(confusion_matrix(yv, predictions))

Entrenamiento: 

              precision    recall  f1-score   support

       False       0.97      1.00      0.99      1837
        True       1.00      0.82      0.90       295

    accuracy                           0.97      2132
   macro avg       0.99      0.91      0.94      2132
weighted avg       0.98      0.97      0.97      2132

Matriz: 

[[1837    0]
 [  54  241]]
Validación: 

              precision    recall  f1-score   support

       False       0.90      0.98      0.94       441
        True       0.87      0.49      0.63        93

    accuracy                           0.90       534
   macro avg       0.89      0.74      0.79       534
weighted avg       0.90      0.90      0.89       534

Matriz: 

[[434   7]
 [ 47  46]]
